# Повний цикл NLP-проекту

## I. Перевірка фактів на достовірність

У межах цієї задачі ви побудуєте систему видобування фактів на правилах, а також інструменти для оцінювання якості роботи цієї системи.

### 1. Домен

Виберіть домен, для якого можна побудувати невелику базу даних на основі [DBPedia](https://dbpedia.org/sparql). Приклади доменів:
- актор і фільми, в яких він знімався;
- письменник і книжки, які він написав;
- музичний гурт і його учасники/концерти/альбоми з роками діяльності/випуску;
- компанія і всі її CEO/CTO з роками діяльності;
- людина і всі її місця роботи з часовими проміжками;
- політик і політичні партії, в яких він брав участь, з роками діяльності;
- винахідник та його винаходи;
- спортсмени і їх команди, матчі, титули тощо.

Проаналізуйте домен і напишіть SPARQL-запит для побудови бази даних.

### 2. Видобування фактів

2.1. Напишіть програму, яка шукає статтю у Вікіпедії про сутність, що належить до вашого домена, та витягає текст цієї статті.

2.2. Напишіть програму, яка опрацьовує текст статті (саме сирий текст, а не таблички, якщо такі є) та витягає з нього інформацію про ваш домен. Цю інформацію ви будете порівнювати зі сформованою базою даних.

### 3. Оцінювання результатів

Розробіть метрику, яка покаже, наскільки інформація, яку ви дістали зі статті, збігається з інформацією в вашій базі даних. Скільки пропущеної інформації? Чи є часткові збіги? (Наприклад, ім'я СЕО збігається лише частково або ім'я СЕО збігається, а роки діяльності різні.)

Додайте ваші спостереження і висновки.

## Приклад

1. Формуємо базу даних про фільми, що знімав Вуді Аллен.

Пишемо SPARQL-запит, щоб отримати всі фільми Вуді Аллена та роки їх випуску:
```
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res:  <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?movie ?year
WHERE {
	?uri dbo:director res:Woody_Allen .
        OPTIONAL {?uri dct:subject ?cat . 
                  ?cat rdfs:label ?year . 
                  FILTER (regex (?year, '\\d+ films', 'i'))} .
	?uri rdfs:label ?movie .
        FILTER (lang(?movie) = 'en')
}
```

Результат:
```
[("Take the Money and Run", "1969 films")
 ("Annie Hall", "1977 films")
 ("Zelig", "1983 films")
 ("The Purple Rose of Cairo", "1985 films")
 ("Alice (1990 film)", "1990 films")
 ("Hollywood Ending", "2002 films")
 ("Match Point", "2005 films")
 ("Everything You Always Wanted to Know About Sex* (*But Were Afraid to Ask) (film)", "1972 films")
 ("Interiors", "1978 films")
 ("Anything Else", "2003 films")
 ("Melinda and Melinda", "2004 films")
 ("Scoop (2006 film)", "2006 films")
 ("Cassandra's Dream", "2007 films")
 ...]
```

Чистимо результат від зайвої інформації (наприклад, треба видалити з назв фільмів інформацію про рік).

2.1. Скрейпимо інформацію про режисера з Вікіпедії: https://en.wikipedia.org/wiki/Woody_Allen.

2.2. Пишемо набір правил (за допомогою NER, частин мови, синтаксичних дерев, регекспів тощо), які витягають потрібну нам інформацію з тексту:

```
Allen directed, starred in, and co-wrote (with Mickey Rose) Take the Money and Run in 1969, which received positive reviews...

Then came two of Allen's most popular films: Annie Hall and Manhattan. Annie Hall (1977) won four Academy Awards, including...
```

3. Розробляємо метрику, оцінюємо результат і намагаємось покращити якість. Пишемо спостереження і висновки.

## II. Курсовий проект

Для свого курсового проекту визначте остаточні метрики і напишіть програму, яка їх реалізує. Покажіть приклад роботи програми на іграшкових даних (до 10 прикладів реальних чи штучних даних).

### Оцінювання

- 80% - перевірка фактів
- 20% - курсовий проект

## Крайній термін

30.03.2019